In [ ]:
import warnings
import datetime as dt
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['M5','M15','H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time

    Src_by          = Src.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)
    Src['T']        = Src_by['Time'].rank().sub(1).astype(int)

    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by2 = Calc.groupby(['Symbol','TF'],        sort=0, group_keys=0)
    Calc_by3 = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    # ================ Full Day ================ #
    Calc['Day Open']            = Calc_by3['Open'].first()                      .reset_index(drop=1)
    Calc['Day High']            = Calc_by3['High'].max()                        .reset_index(drop=1)
    Calc['Day Low']             = Calc_by3['Low'].min()                         .reset_index(drop=1)
    Calc['Day Close']           = Calc_by3['Close'].last()                      .reset_index(drop=1)

    Calc['Day HL']              = Calc['Day High']  - Calc['Day Low']
    Calc['Day Chg']             = Calc['Day Close'] - Calc['Day Open']
    Calc['Day Chg Abs']         = Calc['Day Chg'].abs()

    Calc['Day HL Avg']          = Calc_by2['Day HL']      .expanding().mean()   .reset_index(drop=1)
    Calc['Day Chg Abs Avg']     = Calc_by2['Day Chg Abs'] .expanding().mean()   .reset_index(drop=1)

    Calc['Day HL > Avg']        = Calc['Day HL']      > Calc['Day HL Avg']
    Calc['Day Chg Abs > Avg']   = Calc['Day Chg Abs'] > Calc['Day Chg Abs Avg']


    # ================ Intra Day ================ #
    Calc['Chg Abs']             = Calc['Change'].abs()

    Calc['Chg Abs Avg']         = Calc_by2['Chg Abs'].expanding().mean()        .reset_index(drop=1)
    Calc['HL Avg']              = Calc_by2['HL']     .expanding().mean()        .reset_index(drop=1)

    Calc['Chg Abs > Avg']       = Calc['Chg Abs'] > Calc['Chg Abs Avg'] 
    Calc['HL > Avg']            = Calc['HL']      > Calc['HL Avg']      
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [63]:
def STATS(Calc, T, Col):
    pipe = []
    for TF in ['M5','M15','H1']:
        Df = Calc[(Calc['TF']==TF) & (Calc['T']==T) & (Calc[Col])]
        
        pipe.append({ 'TF':TF,
            'Day Chg Abs > Avg':  np.round(Df['Day Chg Abs > Avg'].mean()*100, 2), 
            'Day HL > Avg':       np.round(Df['Day HL > Avg'].mean()*100, 2), 
        })
    pass
    return pd.DataFrame(pipe)

Sts1 = STATS(Calc, T=0, Col='Chg Abs > Avg')
Sts1

,TF,Day Chg Abs > Avg,Day HL > Avg
0,M5,1.72,1.88
1,M15,0.00,0.00
2,H1,0.00,0.00


In [64]:
Sts2 = STATS(Calc, T=0, Col='HL > Avg')
Sts2

,TF,Day Chg Abs > Avg,Day HL > Avg
0,M5,1.49,1.37
1,M15,0.00,0.00
2,H1,0.00,0.00
